# Proceso de ETL para el dataset de *credits.csv*

---

### Parte 1  |  Desanidación de diccionarios en las columnas **'cast'** y **'crew'**

In [42]:
# Librerias a utilizar
import pandas as pd
import json
import warnings
warnings.filterwarnings('ignore')
import numpy as np

In [2]:
credits = pd.read_csv('../datasets/credits.csv')
credits

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


Como los campos de 'cast' y 'crew' vienen en una lista con diccionarios anidados dentro, creo una funcion para su limpieza

In [ ]:
def clean_dataframe(df_column):
    
    # Crea un diccionario para almacenar los valores limpios y su respectivo 'movie_id'
    values = {'dict':[],'movie_id':[]}
    
    # Itera sobre los valores de la columna junto con sus índices
    for index, value in enumerate(df_column):
        
        value = value.strip('[]') # Elimino los corchetes de cada fila
        
        # Reemplazo las comillas simples por comillas dobles y a los valores None por 'None'
        value = value.replace("'",'"').replace('None','"None"')
        
        # Divide el valor en una lista utilizando el carácter '{' como delimitador
        list_values = value.split('{') # Creo una lista de valores a partir de la llave de apertura ({)
                                       # generando asi en cada valor de la lista un diccionario
        
        # Elimino el primer elemento de la lista, ya que es un valor vacio en vez de ser un diccionario
        list_values = list_values[1:]
        
        # Ahora itero sobre cada valor de 'list_values', siendo cada uno un diccionario
        for i in list_values:
            # Le agrego a cada valor la lave de apertura ya que antes habia utilizado ese caracter para hacer las particiones
            i = '{' + i
            
            # Busco el índice de '}' para tomarlo como cierre de cada diccionario
            cierre = i.index('}')
            
            # Extrae el elemento hasta el cierre '}' y se lo asigno nuevamente a 'i'
            i = i[:cierre + 1]
            
            # Ahora que el valor esta limpio como un diccionario, lo agrego al diccionario 'values' en la clave 'dict'
            values['dict'].append(i)
            
            # A treves del index de la fila, obtengo el 'movie_id' y lo agrego al diccionario en la clave 'movie_id'
            values['movie_id'].append(credits.id.iloc[index])

    # Una vez terminado el loop, creo un dataframe con los valores del diccionario 'values' al que
    # le fui cargando los diccionarios limpios
    df = pd.DataFrame(values)
    
    # Retorno el dataframe
    return df

Ahora con la funcion creada, aplico las transformaciones para la columna 'cast' y para 'crew'. Haciendo esto obtengo un dataframe limpio para cada columna

In [4]:
cast = clean_dataframe(credits.cast)
crew = clean_dataframe(credits.crew)

Creo una nueva columna que me indique si la el valor de la columna 'dict' puede ser transformado en un diccionario o no.

- 1 es que el valor pudo ser transformado en diccionario
- 0 es que por algun error de sintaxis en el valor no se pudo convertir a diccionario y por ende sigue siendo un string

*por default le aplico 1 a cada valor ya que son mas los valores que pueden ser transformados a dict que los que no*

In [5]:
cast['dictionary_format'] = 1
crew['dictionary_format'] = 1

Ahora creo una funcion que trate de convertir cada valor en un diccionario

In [ ]:
def Dictionary_Format(dataframe,column): # Recibe el dataframe completo y la columna a la que se le van a realizar los cambios
    # Itero sobre cada fila del dataframe y su indice
    for index, row in enumerate(column):
        if not isinstance(row, dict): # Verifico si el valor de la fila no es un diccionario
            try:
                # Acá intento cargar el valor de la fila como un diccionario utilizando JSON.loads()
                value = json.loads(row)
                # Si 'value' pudo ser convertido a diccionario efectivamente, se ejecuta lo siguiente:
                dataframe.at[index, 'dict'] = value 
                # A traves de la fila actualizo el valor antiguo que no era diccionario por el valor transformado a dict

            except json.JSONDecodeError as e: 
                # Si hubo un error al tratar de convertir un valor a diccionario, simplemente se setea en la columna
                # 'dictionary_format' con 0.
                dataframe.at[index, 'dictionary_format'] = 0
        else:
            # Si el valor de la fila ya es un diccionario,continuo con el loop
            continue

##### Primero aplico la funcion al nuevo dataframe **'cast'**

In [7]:
Dictionary_Format(cast,cast.dict)
cast.head(10)

,dict,movie_id,dictionary_format
0,"{'cast_id': 14, 'character': 'Woody (voice)', ...",862,1
1,"{'cast_id': 15, 'character': 'Buzz Lightyear (...",862,1
2,"{'cast_id': 16, 'character': 'Mr. Potato Head ...",862,1
3,"{'cast_id': 17, 'character': 'Slinky Dog (voic...",862,1
4,"{'cast_id': 18, 'character': 'Rex (voice)', 'c...",862,1
5,"{'cast_id': 19, 'character': 'Hamm (voice)', '...",862,1
6,"{'cast_id': 20, 'character': 'Bo Peep (voice)'...",862,1
7,"{'cast_id': 26, 'character': 'Andy (voice)', '...",862,1
8,"{'cast_id': 22, 'character': 'Sid (voice)', 'c...",862,1
9,"{'cast_id': 23, 'character': 'Mrs. Davis (voic...",862,1


Visualizo todas las filas con los valores que no pudieron ser modificados a diccionarios por un error de sintaxis o de codificación

In [8]:
cast.loc[cast.dictionary_format == 0]

,dict,movie_id,dictionary_format
46,"{""cast_id"": 1, ""character"": ""Savannah ""Vannah""...",31357,0
47,"{""cast_id"": 2, ""character"": ""Bernadine ""Bernie...",31357,0
48,"{""cast_id"": 3, ""character"": ""Gloria ""Glo"" Matt...",31357,0
95,"{""cast_id"": 52, ""character"": ""Hooker""s Mother""...",949,0
101,"{""cast_id"": 99, ""character"": ""Bosko""s Date"", ""...",949,0
...,...,...,...
562267,"{""cast_id"": 2, ""character"": ""Father O""Neill"", ...",222872,0
562275,"{""cast_id"": 14, ""character"": ""Flashback Dancer...",222872,0
562285,"{""cast_id"": 2, ""character"": ""Aladdin"", ""credit...",325439,0
562363,"{""cast_id"": 2, ""character"": """", ""credit_id"": ""...",106807,0


##### Continuo ahora el el dataframe de **'crew'**

In [9]:
Dictionary_Format(crew,crew.dict)
crew.head()

,dict,movie_id,dictionary_format
0,"{'credit_id': '52fe4284c3a36847f8024f49', 'dep...",862,1
1,"{'credit_id': '52fe4284c3a36847f8024f4f', 'dep...",862,1
2,"{'credit_id': '52fe4284c3a36847f8024f55', 'dep...",862,1
3,"{'credit_id': '52fe4284c3a36847f8024f5b', 'dep...",862,1
4,"{'credit_id': '52fe4284c3a36847f8024f61', 'dep...",862,1


Tambein veo los valores que no se pudieron convertir en diccionario

In [10]:
crew.loc[crew.dictionary_format == 0]

,dict,movie_id,dictionary_format
83,"{""credit_id"": ""589214099251412dc5009d57"", ""dep...",862,0
200,"{""credit_id"": ""56be7b5cc3a36817f400542a"", ""dep...",949,0
337,"{""credit_id"": ""52fe44b79251416c7503e7f5"", ""dep...",12110,0
484,"{""credit_id"": ""59a17819c3a36847d4028802"", ""dep...",5,0
678,"{""credit_id"": ""58810deb9251413028000b42"", ""dep...",451,0
...,...,...,...
462615,"{""credit_id"": ""59ad5583925141070704449b"", ""dep...",339692,0
462669,"{""credit_id"": ""5943b7d6c3a3686c8101f7b6"", ""dep...",462108,0
462924,"{""credit_id"": ""590d7691c3a36864fc00b06e"", ""dep...",329005,0
463710,"{""credit_id"": ""52fe46489251416c9104f725"", ""dep...",37440,0


Ahora, voy a tratar de convertir los valores que no se pudieron convertir anteriormente por algun error de sintaxis

En este caso primero comienzo por 'crew' que tiene mucho menos valores que 'cast'

In [11]:
crew_strings = crew.loc[crew.dictionary_format == 0]

In [12]:
crew_strings.shape[0] # Cantidad de valores que siguen siendo str

3077

In [13]:
crew_strings.head()

,dict,movie_id,dictionary_format
83,"{""credit_id"": ""589214099251412dc5009d57"", ""dep...",862,0
200,"{""credit_id"": ""56be7b5cc3a36817f400542a"", ""dep...",949,0
337,"{""credit_id"": ""52fe44b79251416c7503e7f5"", ""dep...",12110,0
484,"{""credit_id"": ""59a17819c3a36847d4028802"", ""dep...",5,0
678,"{""credit_id"": ""58810deb9251413028000b42"", ""dep...",451,0


Para este proceso, tambien creo una función

In [ ]:
def To_Dictionary(main_df, temporary_df, column): 
    # Recibe el df principal (cast, crew), los dfs temporales (crew_strings, cast_strings) y la columna a la que se le realizan los cambios
    
    # Obtengo la columna especifica del dataframe temporal
    column = temporary_df[column]
    
    # Itero sobre cada valor de la columna y su indice
    for index, row in enumerate(column):
        max_iter = 10 # Esta variable es para definir un limite de intentos para cada valor.
        # Para que no se cree un loop infinito y que se caiga el proceso
        
        # Acá realizo un loop while con la condicion que se ejecute mientras 'max_iter' sea mayor a cero
        while max_iter > 0:
            try:
                # Intenta cargar el valor de la fila como un diccionario nuevamente utilizando JSON.loads()
                value = json.loads(row)
                
                # Si se pudo transformar correctamente, se actualiza el valor en el diccionario principal
                # y ademas se actualiza el valor de la columna 'dictionary_format' a 1 para especificar que ahora es un diccionario
                main_df.at[temporary_df.index[index], 'dict'] = value
                main_df.at[temporary_df.index[index], 'dictionary_format'] = 1
                
                # A la hora de ejecutar la función, me guiaba del siguiente print para ver si el proceso se estaba ejecutando bien
                # o si el loop se habia trabado o caido
                #>>> print('Success',index,value)
                
                break  # Luego de que lo anterior se haya realizado, rompo el loop para continuar con el proximo valor
            
            except json.JSONDecodeError as e:
                # En caso de un error, busco en que posición se produjo el error gracias al metodo .pos
                # y le quito el caracter de la posicion del error a la fila
                pos = e.pos - 1
                row = row[:pos] + row[pos+1:]
                
                # Tambien para ir tanteando si funcionaba la parte del 'except', hice lo mismo que arriba
                #>>> print('Fail',index,row)
                
                max_iter -= 1 # Le elimino un intento a la variable de 'max_iter'
                continue # Este continue hace que se reproduzca de nuevo el while en el mismo elemento pero con un intento menos

Aplico la funcion para el dataframe de 'crew'

In [ ]:
To_Dictionary(crew,crew_strings, 'dict') 

In [16]:
crew.head(10)

,dict,movie_id,dictionary_format
0,"{'credit_id': '52fe4284c3a36847f8024f49', 'dep...",862,1
1,"{'credit_id': '52fe4284c3a36847f8024f4f', 'dep...",862,1
2,"{'credit_id': '52fe4284c3a36847f8024f55', 'dep...",862,1
3,"{'credit_id': '52fe4284c3a36847f8024f5b', 'dep...",862,1
4,"{'credit_id': '52fe4284c3a36847f8024f61', 'dep...",862,1
5,"{'credit_id': '52fe4284c3a36847f8024f67', 'dep...",862,1
6,"{'credit_id': '52fe4284c3a36847f8024f6d', 'dep...",862,1
7,"{'credit_id': '52fe4284c3a36847f8024f73', 'dep...",862,1
8,"{'credit_id': '52fe4284c3a36847f8024f79', 'dep...",862,1
9,"{'credit_id': '52fe4284c3a36847f8024f8b', 'dep...",862,1


Si ahora accedo a algun valor random, veo que efectivamente es un diccionario con sus respectivas claves y valores. Por lo que la funcion logro convertir efectivamente todos los valores a diccionarios

In [17]:
crew.dict.head(10)[0]

{'credit_id': '52fe4284c3a36847f8024f49',
 'department': 'Directing',
 'gender': 2,
 'id': 7879,
 'job': 'Director',
 'name': 'John Lasseter',
 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}

Ahora ya no hay ninguna fila que tenga 0 en la columna 'dictionary_format', lo que quiere decir que ya no hay strings si no solamente diccionarios

In [18]:
crew.loc[crew.dictionary_format == 0].count()

dict                 0
movie_id             0
dictionary_format    0
dtype: int64

In [19]:
crew.loc[crew.dictionary_format == 0]

,dict,movie_id,dictionary_format


##### Realizo el mismo procedimiento para el dataframe de 'cast'

Primero quiero eliminar las barras invertidas ('\\') para evitar errores

In [21]:
cast_strings['dict'] = cast_strings['dict'].apply(lambda x: x.replace('\\', ''))

In [20]:
cast_strings = cast.loc[cast.dictionary_format == 0]
cast_strings.shape[0] # Todos los valores que no se pudieron convertir anteriormente a diccionario

28488

In [22]:
cast.head()

,dict,movie_id,dictionary_format
0,"{'cast_id': 14, 'character': 'Woody (voice)', ...",862,1
1,"{'cast_id': 15, 'character': 'Buzz Lightyear (...",862,1
2,"{'cast_id': 16, 'character': 'Mr. Potato Head ...",862,1
3,"{'cast_id': 17, 'character': 'Slinky Dog (voic...",862,1
4,"{'cast_id': 18, 'character': 'Rex (voice)', 'c...",862,1


In [ ]:
To_Dictionary(cast, cast_strings,'dict')

Al igual que con el dataframe de 'crew', los valores de 'cast' ahora son todos diccionarios

In [25]:
cast.loc[cast.dictionary_format == 0].count()

dict                 0
movie_id             0
dictionary_format    0
dtype: int64

In [26]:
cast_strings = cast.loc[cast.dictionary_format == 0]
cast_strings.count()

dict                 0
movie_id             0
dictionary_format    0
dtype: int64

#### Demostración de los valores de los dataframes actuales

In [28]:
cast.head()

,dict,movie_id,dictionary_format
0,"{'cast_id': 14, 'character': 'Woody (voice)', ...",862,1
1,"{'cast_id': 15, 'character': 'Buzz Lightyear (...",862,1
2,"{'cast_id': 16, 'character': 'Mr. Potato Head ...",862,1
3,"{'cast_id': 17, 'character': 'Slinky Dog (voic...",862,1
4,"{'cast_id': 18, 'character': 'Rex (voice)', 'c...",862,1


In [29]:
crew.head()

,dict,movie_id,dictionary_format
0,"{'credit_id': '52fe4284c3a36847f8024f49', 'dep...",862,1
1,"{'credit_id': '52fe4284c3a36847f8024f4f', 'dep...",862,1
2,"{'credit_id': '52fe4284c3a36847f8024f55', 'dep...",862,1
3,"{'credit_id': '52fe4284c3a36847f8024f5b', 'dep...",862,1
4,"{'credit_id': '52fe4284c3a36847f8024f61', 'dep...",862,1


Exporto los dataframes como archivos csv para que cuando quiera volver a correr todo el notebook no se hagan todos los procesos de nuevo. Simplemente cargo con pandas los csv y listo

In [30]:
cast.to_csv('../preprocessing_data/cast.csv',sep='|')
crew.to_csv('../preprocessing_data/crew.csv',sep='|')

---

### Parte 2  |  Limpieza y transformación de los nuevos dataframes

En esta segunda parte del proceso de ETL, lo que haré es obtener información de cada diccionario de los dataframes 'cast' y 'crew' para poder conformar un dataframe unico con toda la información organizada.

Esto es posible ya que ahora, gracias a que los elementos tienen un formato de diccionario, se puede acceder facilmente a los valores y claves de los mismos

In [31]:
cast = pd.read_csv('../preprocessing_data/cast.csv',sep='|')
# Como olvide poner index=False al momento de guardar los csv's, cuando los cargo, se incluye la columna Unnamed: 0 con los indices
# Elimino 'dictionary_format' ya que anteriormente me asegure de que todos eran diccionarios y tambien elimino la columna Unnamed: 0
cast = cast.drop(columns={'Unnamed: 0','dictionary_format'}) 
cast.head(3)

,dict,movie_id
0,"{'cast_id': 14, 'character': 'Woody (voice)', ...",862
1,"{'cast_id': 15, 'character': 'Buzz Lightyear (...",862
2,"{'cast_id': 16, 'character': 'Mr. Potato Head ...",862


In [32]:
crew = pd.read_csv('../preprocessing_data/crew.csv',sep='|')
crew = crew.drop(columns={'Unnamed: 0','dictionary_format'})
crew.head(3)

,dict,movie_id
0,"{'credit_id': '52fe4284c3a36847f8024f49', 'dep...",862
1,"{'credit_id': '52fe4284c3a36847f8024f4f', 'dep...",862
2,"{'credit_id': '52fe4284c3a36847f8024f55', 'dep...",862


#### Elimino las filas duplicadas de los dataframes

In [33]:
print('Cast antes',cast.shape[0])
cast = cast.drop_duplicates(subset='dict')
print('Cast despues',cast.shape[0])

Cast antes 562474
Cast despues 562044


In [34]:
print('Crew antes',crew.shape[0])
crew = crew.drop_duplicates(subset='dict')
print('Crew despues',crew.shape[0])

Crew antes 464314
Crew despues 463836


Para poder extraer la información de cada diccionario, primero debo convertirlos a dict con json.loads() ya que al momento de cargar los nuevos csv's vienen por default como 'dtype=object'

In [35]:
# Reemplazo las comillas simples por comillas dobles porque json.loads() lo requiere
cast.dict = cast.dict.apply(lambda x: json.loads(x.replace("'",'"')))
crew.dict = crew.dict.apply(lambda x: json.loads(x.replace("'",'"')))

Hago una lista con todas las claves que tienen los diccionarios menos 'profile_path' que no me va a ser util

Como luego expandiré el dataframe, es decir que crearé nuevas columnas, veo cuales son las columnas que tienen en comun los diccionarios de cada dataframe.

In [36]:
print('Cast     ',cast.dict[0].keys())
print('Crew     ',crew.dict[0].keys())

Cast      dict_keys(['cast_id', 'character', 'credit_id', 'gender', 'id', 'name', 'order', 'profile_path'])
Crew      dict_keys(['credit_id', 'department', 'gender', 'id', 'job', 'name', 'profile_path'])


    Proximas columnas de los dataframes

In [37]:
new_columns_cast = ['cast_id', 'character', 'credit_id', 'gender', 'id', 'name', 'order', 'profile_path']
new_columns_crew = ['credit_id', 'department', 'gender', 'id', 'job', 'name', 'profile_path']

Creo las nuevas columnas para cada dataframe

In [38]:
column_fails_cast = [] # Para las claves que no se pudieron crear como columna
# Itero las lista que cree arriba
for k in new_columns_cast:
    try: # Intenta crear la nueva columna en 'cast' y accede a la clave en el diccionario para obtener el valor y asi ponerlo en la nueva columna
        cast[k] = cast.dict.apply(lambda x: x[k])
    except KeyError: # Si falla, que lo agregue a la lista de 'columns_fails_cast'
        column_fails_cast.append(k)


# Mismo proceso para 'crew'
column_fails_crew = []
for k in new_columns_crew:
    try:
        crew[k] = crew.dict.apply(lambda x: x[k])
    except KeyError:
        column_fails_crew.append(k)

#### Resultado del procedimiento anterior:

In [39]:
cast.head(3)

,dict,movie_id,cast_id,character,gender,id,name,order,profile_path
0,"{'cast_id': 14, 'character': 'Woody (voice)', ...",862,14,Woody (voice),2,31,Tom Hanks,0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,"{'cast_id': 15, 'character': 'Buzz Lightyear (...",862,15,Buzz Lightyear (voice),2,12898,Tim Allen,1,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,"{'cast_id': 16, 'character': 'Mr. Potato Head ...",862,16,Mr. Potato Head (voice),2,7167,Don Rickles,2,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg


In [40]:
crew.head(3)

,dict,movie_id,credit_id,department,gender,id,job,name,profile_path
0,"{'credit_id': '52fe4284c3a36847f8024f49', 'dep...",862,52fe4284c3a36847f8024f49,Directing,2,7879,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,"{'credit_id': '52fe4284c3a36847f8024f4f', 'dep...",862,52fe4284c3a36847f8024f4f,Writing,2,12891,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,"{'credit_id': '52fe4284c3a36847f8024f55', 'dep...",862,52fe4284c3a36847f8024f55,Writing,2,7,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg


Como habia dicho antes, gracias a que la información este comprimida en diccionarios, es muy facil despues organizarla por columnas como lo que acabo de hacer

In [41]:
print('Columnas que dieron error en cast',column_fails_cast)
print('Columnas que dieron error en crew',column_fails_crew)

Columnas que dieron error en cast ['credit_id']
Columnas que dieron error en crew []


In [43]:
# Creo la columan de credit_id vacia
cast['credit_id'] = np.nan

# Reliazo un procedimiento parecido pero para la columna 'credit_id'
indexes_fail = [] # Una lista para alojar los indices que no se pudieron agregar a la columna 'credit_id'

# Itero la columna 'dict', obteniendo el index de la fila y el valor
for index, dicc in enumerate(cast.dict):
    try: # Intento acceder a cada valor de la clave 'credit_id' del diccionario en 'dict'
        # y una vez extraido ese valor lo inserto en la nueva columna 'credit_id'
        cast.at[index, 'credit_id'] = dicc['credit_id']
    except KeyError: # Si algun valor falla, que lo inserte en la lista creada arriba
        indexes_fail.append(index)

In [44]:
indexes_fail # Indices de los valores que fallaron

[135724, 498466, 537045]

Actualizo manualmente estos valores ya que son solo tres filas

In [45]:
for i in cast.iloc[indexes_fail].dict:
    print(i)

{'cast_id': 5, 'character': 'Clips from For Me and My Gal', 'Easter Parade, & Girl Crazyetc (archive footage), credit_id': '567807ecc3a3681689008d0e', 'gender': 1, 'id': 9066, 'name': 'Judy Garland', 'order': 2, 'profile_path': '/r2p0SpwpkDc5hhg7hqbar9bKMox.jpg'}
{'cast_id': 1, 'character': 'Private detective Kogorô Akechi (2x); A man; Masaki (story Imomushi', 'Kagami jigoku, Kasei no unga, and Mushi), credit_id': '52fe4731c3a36847f81285ab', 'gender': 2, 'id': 13275, 'name': 'Tadanobu Asano', 'order': 0, 'profile_path': '/zlZsST8s1Apm6D6bCyYeWrCKZCy.jpg'}
{'cast_id': 5, 'character': 'Erkki Uolevi Lahti, Dynamiitti-Lahti', 'Tyny, credit_id': '58aac76092514141a0001ddf', 'gender': 2, 'id': 110772, 'name': 'Harri Hyttinen', 'order': 6, 'profile_path': 'None'}


Como se puede ver arriba, hubo un mal interpretamiento en las claves ya que cada fila tiene comas en los valores de 'character', corrompiendo asi la clave de 'credit_id'.

In [46]:
# Estos son los valores reales de la clave 'credit_id'
credits_id_fixed = ['567807ecc3a3681689008d0e','52fe4731c3a36847f81285ab','58aac76092514141a0001ddf']

Actualizo estos mismos de una manera rapida. 

In [51]:
cast.at[135733, 'credit_id'] = '567807ecc3a3681689008d0e'
cast.at[498861, 'credit_id'] = '52fe4731c3a36847f81285ab'
cast.at[537459, 'credit_id'] = '58aac76092514141a0001ddf'

In [56]:
cast.head(3)

,dict,movie_id,cast_id,character,gender,id,name,order,profile_path,credit_id
0,"{'cast_id': 14, 'character': 'Woody (voice)', ...",862.0,14.0,Woody (voice),2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,52fe4284c3a36847f8024f95
1,"{'cast_id': 15, 'character': 'Buzz Lightyear (...",862.0,15.0,Buzz Lightyear (voice),2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg,52fe4284c3a36847f8024f99
2,"{'cast_id': 16, 'character': 'Mr. Potato Head ...",862.0,16.0,Mr. Potato Head (voice),2.0,7167.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg,52fe4284c3a36847f8024f9d


Ahora con todas las nuevas columnas creadas, elimino 'profile_path' y 'dict' ya que no las necesito

In [57]:
cast = cast.drop(columns={'dict','profile_path'})
crew = crew.drop(columns={'dict','profile_path'})

#### Con estos cambios y transformaciones, se puede ver que la información esta mucho mas organizada y limpia, obteniendo en cada campo un valor en especifico en vez de tener un diccionario entero

In [58]:
cast.head()

,movie_id,cast_id,character,gender,id,name,order,credit_id
0,862.0,14.0,Woody (voice),2.0,31.0,Tom Hanks,0.0,52fe4284c3a36847f8024f95
1,862.0,15.0,Buzz Lightyear (voice),2.0,12898.0,Tim Allen,1.0,52fe4284c3a36847f8024f99
2,862.0,16.0,Mr. Potato Head (voice),2.0,7167.0,Don Rickles,2.0,52fe4284c3a36847f8024f9d
3,862.0,17.0,Slinky Dog (voice),2.0,12899.0,Jim Varney,3.0,52fe4284c3a36847f8024fa1
4,862.0,18.0,Rex (voice),2.0,12900.0,Wallace Shawn,4.0,52fe4284c3a36847f8024fa5


In [59]:
crew.head()

,movie_id,credit_id,department,gender,id,job,name
0,862,52fe4284c3a36847f8024f49,Directing,2,7879,Director,John Lasseter
1,862,52fe4284c3a36847f8024f4f,Writing,2,12891,Screenplay,Joss Whedon
2,862,52fe4284c3a36847f8024f55,Writing,2,7,Screenplay,Andrew Stanton
3,862,52fe4284c3a36847f8024f5b,Writing,2,12892,Screenplay,Joel Cohen
4,862,52fe4284c3a36847f8024f61,Writing,0,12893,Screenplay,Alec Sokolow


Finalmente, con toda la data limpia y organizada como corresponde, procedo a exportar los dataframe como archivos 'csv' para su posterior tratamiento

In [60]:
cast.to_csv('../clean_data/final_cast.csv',sep=',',index=False)
crew.to_csv('../clean_data/final_crew.csv',sep=',',index=False)